The Cells from `Connecting the GPU` to `Installing VSGAN` are required. They must also be run in the shown order.

Yuuno is a convenience IPython module but it is not required to be used. You may simply use vspipe instead.

In [ ]:
#@title Connect to GPU
#@markdown Enable GPU in Runtime -> Change runtime type first
gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
print("GPU:", gpu[1] or "None")
print("The Tesla T4 and P100 are fast and support hardware encoding. The K80 and P4 are slower.")
print("Sometimes resetting the instance in the 'runtime' tab will give you a different GPU.")

In [ ]:
#@title Install VapourSynth and VSGAN
#@markdown This does not install ANY scripts or plugins, just VapourSynth.
# VapourSynth is not available in any repositories that are currently supported.
# Therefore we use a pre-compiled copy of VapourSynth and manually copy the files in.
print("Installing VapourSynth C++ Library")
!mkdir VapourSynth-61-linux_x86_64
!mkdir -p /usr/local/lib/vapoursynth/
!mkdir -p /usr/local/lib/x86_64-linux-gnu/
%cd VapourSynth-61-linux_x86_64
!wget https://github.com/rlaphoenix/install-vapoursynth-action/releases/download/v1.0.0/VapourSynth-61-linux_x86_64.7z
!7z x VapourSynth-61-linux_x86_64.7z
!cp usr_local_bin/vspipe /usr/local/bin/vspipe
!mv usr_local_lib/* /usr/local/lib
!sudo ldconfig
!chmod +x /usr/local/bin/vspipe
print("Installing VapourSynth Python Interface")
%pip install VapourSynth==61
print("Installing VSGAN")
%pip install vsgan

In [ ]:
#@title Mount Google Drive to /content/drive to load models or videos 
import os
from google.colab import drive

if not os.path.isdir("/content/drive"):
  drive.mount("/content/drive", force_remount=True)

In [ ]:
#@title Install x264
!apt-get install -y --allow-unauthenticated x264

In [ ]:
#@title Install Yuuno
#@markdown This is an optional convenience feature, allowing IPython % magic commands.
%pip install yuuno
!yuuno jupyter install

In [ ]:
#@title After installing Yuuno, reboot (NOT RESET!) the current instance then start this.
#@markdown For some reason it thinks yuuno isn't an IPython module if you don't first restart.
%load_ext yuuno

In [ ]:
#@title Start an example encode and download it
%%vspipe --y4m | x264 - --demuxer y4m -o example.mkv
from vsgan import ESRGAN
clip = core.std.BlankClip(width=854, height=480, color=[0, 255, 0], format=vs.RGB24)
clip = ESRGAN(clip, device="cuda").\
    load("/content/drive/My Drive/Models/2X_DigitalFilmV5_Lite.pth").\
    apply().\
    clip
clip = core.resize.Spline16(clip, format=vs.YUV420P8, matrix=1)
clip.set_output()

In [ ]:
#@title Download the example encode
from google.colab import files
files.download("example.mkv")